In [ ]:
import pandas as pd
from utils import *

In [ ]:
market_returns = load_pkl('market_returns.pkl')
# funds_returns = load_pkl('funds_returns.pkl')
funds_returns = load_pkl('data/data_frame_complete.pkl')
funds_returns.rename(columns={'caldt':'date'}, inplace=True)
funds_returns['date'] = pd.to_datetime(funds_returns.date)

In [ ]:
df = pd.merge(funds_returns, market_returns, on='date')
df['year'] = df.date.apply(lambda date: date.year)

# add yearly returns per fund
df.mret +=1
df_yearly_fund_returns = df.groupby(['comp_cik', 'year']).prod().reset_index()[['year', 'comp_cik', 'mret']]
df_yearly_fund_returns.rename(columns={'mret':'yret'}, inplace=True)
df_yearly_fund_returns.yret -=1

df.mret -=1

# compute beta
cov = df.groupby(['comp_cik', 'year']).cov().iloc[0::2,-1]
var = df.groupby(['comp_cik', 'year']).vwretd.var()
df = df.groupby(['comp_cik', 'year']).first().reset_index().drop(columns='mret')
df['beta'] = cov.reset_index()['vwretd']/var.reset_index()['vwretd']
df

# add risk free rate
risk_free = load_pkl('risk_free.pkl')
risk_free.rename(columns={"tmytm": "risk_free"}, inplace=True)
df = pd.merge(df, risk_free, on='year')
df = pd.merge(df, df_yearly_fund_returns, on=['year', 'comp_cik'])
# check
# df[df.mcaldt != df.date]
df

In [ ]:
df['basis_points_delta'] = df.yret - df.vwretd


In [ ]:
df[['comp_cik', 'year', 'vwretd', 'beta', 'risk_free', 'yret', 'basis_points_delta']]

In [ ]:
df.comp_cik = df.comp_cik.astype(int)

In [ ]:
finbert_predictions = pd.read_csv('df_finbert_predictions.csv', index_col=0)
finbert_predictions = finbert_predictions[['cik', 'year', 'finbert_positive', 'finbert_negative', 'finbert_neutral']]
finbert_predictions.rename(columns={'cik':'comp_cik'}, inplace=True)

In [ ]:
df_final = pd.merge(df[['comp_cik', 'year', 'vwretd', 'yret', 'basis_points_delta']],
         finbert_predictions[['comp_cik', 'year', 'finbert_positive', 'finbert_negative', 'finbert_neutral']],
         on=['comp_cik', 'year'])

In [ ]:
df_final.to_clipboard()

In [ ]:
%load_ext ipython_clipboard

In [ ]:
%pickle df_final

In [ ]:
df_final.to_csv('df_final.csv')

In [ ]:
%clip
finbert_predictions.describe().drop(columns=['year', 'comp_cik']).to_latex(float_format='%.3f', bold_rows=True, longtable=True, caption='Finbert prediction output')

In [ ]:
%clip
finbert_predictions.describe().drop(columns=['year', 'comp_cik']).to_latex(float_format='%.3f', bold_rows=True, longtable=True, caption='Finbert prediction output')